In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import sys
sys.path.append('../../')
from bert import nli
import tensorflow as tf
import re

/home/link/miniconda3/envs/bert/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/link/miniconda3/envs/bert/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/link/miniconda3/envs/bert/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/link/miniconda3/envs/bert/lib/python3.6/site-pack

In [2]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

nli_labels = ["contradiction", "entailment", "neutral"]
rumor_labels = ["rumor", "truth", "unconfirmed claim","nonrumor"]

In [3]:
tf.logging.set_verbosity(tf.logging.ERROR)
predict_fn = nli.get_nli_predict_fn(
    bert_base_dir = '../../chinese-bert_chinese_wwm_L-12_H-768_A-12',
    output_dir = '../../xnli_output'
)

In [4]:
model = SentenceTransformer('distiluse-base-multilingual-cased')

/home/link/miniconda3/envs/bert/lib/python3.6/site-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


In [5]:
dxy= pd.read_csv('../data/DXY_rumors.csv')
piyao = pd.read_csv('../data/piyao.csv')
df = pd.concat([dxy,piyao])
# df = df.drop(['_id','id','mainSummary','summary','sourceUrl','crawlTime','body'],axis=1)

In [6]:
def detect_rumor(input_text):
    text_b = input_text
    x = np.array(model.encode([text_b]))
    z = np.array(model.encode(df.title.values))
    f=z.dot(x.transpose()).reshape(len(df))
    p=softmax(f)
    most_match_index = np.argmax(p)
    text_a = re.sub('[？?]','',df.title.values[most_match_index])
    label_a = df.rumorType.values[most_match_index]
    #df.values[np.where(p>0.004)]
    #print(text_a,rumor_labels[label_a])
    examples = [
        nli.InputExample(guid='prodiction-1',text_a=text_a,text_b=text_b,label='neutral')
    ]
    result = predict_fn(examples)
    for (i, prediction) in enumerate(result):
        probabilities = prediction['probabilities']
        print(text_a,'->',text_b,': ',nli_labels[np.argmax(probabilities)])
        if np.argmax(probabilities)==0:
            rumor_labels = ["真相", "谣言", "误区","新闻"]
        else:
            rumor_labels = ["谣言", "真相", "误区","新闻"]
        if np.argmax(probabilities)<2:
            print(text_b+'是'+rumor_labels[label_a])
        else:
            print('最相关的'+rumor_labels[label_a]+'是',text_a)


In [7]:
detect_rumor('钟南山没被传染')

钟南山院士被传染 -> 钟南山没被传染 :  contradiction
钟南山没被传染是真相


In [14]:
detect_rumor('八旬妻子球场上边投篮边等钟南山回家')

钟南山：全国人民在家隔离两周 -> 八旬妻子球场上边投篮边等钟南山回家 :  neutral
最相关的谣言是 钟南山：全国人民在家隔离两周


In [9]:
def get_documents(input_text):
    text_b = input_text
    x = np.array(model.encode([text_b]))
    z = np.array(model.encode(df.title.values))
    f=z.dot(x.transpose()).reshape(len(df))
    p=softmax(f)
    return sorted([*zip(p,df.title.values,df.mainSummary.values,df.rumorType.values)], key=lambda x:x[0], reverse=True)[0:20]

In [10]:
get_documents('“100多人带着病毒回来了”发生在黑龙江黑河')

[(0.0014822732, '黑龙江黑河“100多人带着病毒回来了”？', nan, 0),
 (0.0010567, '病毒通过衣物传染人的几率极低', nan, 1),
 (0.0010141345, '吉林省和龙市出现一例新冠病毒感染者？', nan, 0),
 (0.0010084858, '网传感染病例在10万例左右？', '国家卫健委回应：不实消息', 0),
 (0.0010038397, '黑龙江省疫情数据翻倍？', nan, 0),
 (0.0009992061, '黑龙江返银川一家被确诊？', nan, 0),
 (0.0009988224, '打工男子恶意将新冠病毒从湖北带回河南安阳？', nan, 0),
 (0.0009954633, '天津王顶堤批发市场检出十几个新冠病毒携带者？', nan, 0),
 (0.0009898172, '外地游客因新冠病毒感染在重庆渝中死亡？', nan, 0),
 (0.0009872626, '黑龙江新冠肺炎隔离点突发大火坍塌，10人死亡70多人遇难？', nan, 0),
 (0.0009827306, '我国新冠病毒灭活疫苗获批进入临床试验', nan, 1),
 (0.00097555417, '病毒会漂浮在空气中形成“病毒云”？', nan, 0),
 (0.0009754756, '利川市政府召开关于对疫情发起总攻的紧急会议？', nan, 0),
 (0.00097298896, '武汉病毒所研究生为“零号病人” ？', nan, 0),
 (0.0009698452, '湖南益阳一名武大学生传染爷爷新冠病毒并接触790人？', nan, 0),
 (0.0009682716, '国家卫健委：湖北以外地区全面有序恢复正常的疫苗接种', nan, 1),
 (0.0009677782, '北京用大数据锁定35万人筛查病毒？', nan, 0),
 (0.00096657226, '新冠病毒会通过下水道传播？', '丁香医生团队：不能排除有这一可能', 2),
 (0.00096503407, '河北省多地发布疫情紧急通知？', nan, 0),
 (0.0009562716, '病毒在头发上滞留的可能性非常低', nan, 1)]

In [99]:
import json
a=json.dumps([x[0] for x in get_documents('双黄连可以预防新冠病毒')])

In [106]:
zip(df.title.values,df.mainSummary.values,df.rumorType.values,p)

In [113]:
piyao

,Unnamed: 0,title,pubtime,linkurl,rumorType,editor,author,sourcename
0,0,广西有人3次检测呈阳性？,2020-06-28 10:49:26,http://www.piyao.org.cn/2020-06/28/c_121067970...,0,张智萍,NaN,昭平公安
1,1,蚊蝇可以传播新冠病毒？,2020-06-28 10:49:51,http://www.piyao.org.cn/2020-06/28/c_121067958...,0,张智萍,NaN,中国新闻网
2,2,南昌民德路锦江之星酒店发现有俄罗斯人患新冠肺炎？,2020-06-26 11:05:25,http://www.piyao.org.cn/2020-06/26/c_121067743...,0,张智萍,NaN,央广网
3,4,市民路过中高风险地区健康码会变色？,2020-06-24 15:44:27,http://www.piyao.org.cn/2020-06/24/c_121067535...,0,冯栋,张璐,新京报
4,5,上海30天内禁止跨省旅游团？,2020-06-24 16:28:01,http://www.piyao.org.cn/2020-06/24/c_121067536...,0,冯栋,白璐,上海网络辟谣
...,...,...,...,...,...,...,...,...
943,995,中国向菲律宾捐赠20万只口罩？,2020-02-11 23:21:37,http://www.piyao.org.cn/2020-02/11/c_121046411...,0,张智萍,NaN,中国驻菲律宾大使馆官网
944,996,疫情致广西防城港市，玉林市博白县、陆川县米粮断供？,2020-02-12 08:42:01,http://www.piyao.org.cn/2020-02/12/c_121047075...,0,张智萍,NaN,防城港市市场监管局
945,997,黑龙江省疫情数据翻倍？,2020-02-11 22:55:06,http://www.piyao.org.cn/2020-02/11/c_121046876...,0,张智萍,NaN,东北网
946,998,长沙封城、株洲封外地户口进不了城，一律凭身份证进城？,2020-02-12 08:36:35,http://www.piyao.org.cn/2020-02/12/c_121047076...,0,张智萍,NaN,湖南省长沙市交警支队官方微博


In [120]:
detect_rumor('连花清瘟能治疗新冠病毒')

连花清瘟能够有效提高新冠病毒临床治愈率 -> 连花清瘟能治疗新冠病毒 :  entailment
连花清瘟能治疗新冠病毒是真相


In [122]:
detect_rumor('连花清瘟对治疗新冠病毒有效')

连花清瘟能够有效提高新冠病毒临床治愈率 -> 连花清瘟对治疗新冠病毒有效 :  entailment
连花清瘟对治疗新冠病毒有效是真相


In [123]:
detect_rumor('连花清瘟对治疗新冠病毒无效')

连花清瘟能够有效提高新冠病毒临床治愈率 -> 连花清瘟对治疗新冠病毒无效 :  contradiction
连花清瘟对治疗新冠病毒无效是谣言


# For conclusion part

In [112]:
def detect(input_text):
    text_b = input_text
    x = np.array(model.encode([text_b]))
    z = np.array(model.encode(df.title.values))
    f=z.dot(x.transpose()).reshape(len(df))
    p=softmax(f)
    most_match_index = np.argmax(p)
    text_a = re.sub('[？?]','',df.title.values[most_match_index])
    label_a = df.rumorType.values[most_match_index]
    #df.values[np.where(p>0.004)]
    #print(text_a,rumor_labels[label_a])
    examples = [
        nli.InputExample(guid='prodiction-1',text_a=text_a,text_b=text_b,label='neutral')
    ]
    result = predict_fn(examples)
    for (i, prediction) in enumerate(result):
        probabilities = prediction['probabilities']
        print(text_a,'->',text_b,': ',nli_labels[np.argmax(probabilities)])
        return [text_a, np.argmax(probabilities)]

def search(input_text):
    text_b = input_text
    x = np.array(model.encode([text_b]))
    z = np.array(model.encode(df.title.values))
    f=z.dot(x.transpose()).reshape(len(df))
    p=softmax(f)
    most_match_index = np.argmax(p)
    text_a = re.sub('[？?]','',df.title.values[most_match_index])
    label_a = df.rumorType.values[most_match_index]
    return text_a

def detect_with_document(input_text, text_a):
    text_b = input_text
    examples = [
        nli.InputExample(guid='prodiction-1',text_a=text_a,text_b=text_b,label='neutral')
    ]
    result = predict_fn(examples)
    for (i, prediction) in enumerate(result):
        probabilities = prediction['probabilities']
        print(text_a,'->',text_b,': ',nli_labels[np.argmax(probabilities)])
        return [text_a, np.argmax(probabilities)]


In [113]:
search('武汉市解除离汉离鄂通道管控措施')

'4月8日零时起，武汉市解除离汉离鄂通道管控措施'

In [146]:
tdnli= pd.read_csv('../data/tinydata_nli_result.csv')
tdnli.drop(['_id'],inplace=True,axis=1)
# tdnli.drop(tdnli.columns[[0,7,8,9,10,11]],axis=1,inplace=True)
# tdnli.to_csv('../data/tinydata_nli.csv')

In [116]:
tdnli.to_csv('../data/tinydata_nli_result.csv')

In [115]:
tdnli['entailment_document'] = -1
tdnli['entailment_result'] = -1
tdnli['contradiction_document'] = -1
tdnli['contradiction_result'] = -1
tdnli['neutral_document'] = -1
tdnli['neutral_result'] = -1
tdnli.head(3)

,id,label,document,entailment,contradiction,neutral,entailment_document,entailment_result,contradiction_document,contradiction_result,neutral_document,neutral_result
0,390,5,黑龙江黑河“100多人带着病毒回来了”,“100多人带着病毒回来了”发生在黑龙江黑河,黑龙江黑河没有“100多人带着病毒回来了”,重走抗联路!吉林省委党史研究室率专家一行实地踏查老黑河,-1,-1,-1,-1,-1,-1
1,241,7,钟南山预测“5月疫情将全面爆发”,“5月疫情将全面爆发”被钟南山预测,钟南山没有预测“5月疫情将全面爆发”,八旬妻子球场上边投篮边等钟南山回家,-1,-1,-1,-1,-1,-1
2,867,7,病毒会漂浮在空气中形成“病毒云”,“病毒云”是由漂浮在空气中的病毒形成,病毒不会漂浮在空气中形成“病毒云”,米兰官方:皮奥利新冠病毒检测呈阳性,-1,-1,-1,-1,-1,-1


In [147]:
for i, row in tdnli.iterrows():
    for test_label in ['entailment', 'contradiction', 'neutral']:
        if row[test_label+'_document']=='-1':
            if test_label=='neutral':
                result = detect_with_document(row[test_label], row['document'])
            else:
                result[0] = search(row[test_label])
                result[1] = detect_with_document(row[test_label], row['document'])[1]
            tdnli.loc[i, test_label+'_document'] = result[0]
            tdnli.loc[i, test_label+'_result'] = result[1]
            tdnli.to_csv('../data/tinydata_nli_result.csv')
            print(i,row[test_label+'_document'])

自来水采用上限标准加氯消毒用水时至少静置两小时 -> 加氯消毒用水时至少静置两小时当自来水采用上限标准 :  entailment
103 -1
自来水采用上限标准加氯消毒用水时至少静置两小时 -> 自来水没有采用上限标准加氯消毒用水时至少静置两小时 :  contradiction
103 -1
自来水采用上限标准加氯消毒用水时至少静置两小时 -> 不用水,表也转,“被用水”让居民很抓狂 :  contradiction
103 -1
重庆部分区县加油站限量供应 -> 加油站限量供应出现在重庆部分区县 :  entailment
104 -1
重庆部分区县加油站限量供应 -> 重庆部分区县加油站无限量供应 :  contradiction
104 -1
重庆部分区县加油站限量供应 -> 今天早上8点,2020重庆马拉松激情开跑 :  contradiction
104 -1
市民路过中高风险地区健康码会变色 -> 健康码会因为市民路过中高风险地区而变色 :  entailment
105 -1
市民路过中高风险地区健康码会变色 -> 市民路过中高风险地区健康码不会变色 :  contradiction
105 -1
市民路过中高风险地区健康码会变色 -> 现在进学校都需要看健康码的颜色。 :  neutral
105 -1
社区共用体温计有风险或有交叉感染！ -> 交叉感染可能会通过社区共用体温计发生。 :  entailment
106 -1
社区共用体温计有风险或有交叉感染！ -> 社区共用体温计，没有交叉感染的风险。 :  contradiction
106 -1
社区共用体温计有风险或有交叉感染！ -> 社区共用体温计给社区中的人带来便利。 :  contradiction
106 -1
江西九江长江一桥解除封闭 -> 解除封闭发生在江西九江长江一桥 :  entailment
107 -1
江西九江长江一桥解除封闭 -> 江西九江长江一桥没有解除封闭 :  contradiction
107 -1
江西九江长江一桥解除封闭 -> 九江区域航运中心建设扬帆远航 :  contradiction
107 -1
进沪前14天内有过这8个国家旅行或居住史一律隔离14天 -> 进沪前14天内一律隔离14天的人包括有过这8个国家旅行或居住史的人 :  entailm

In [ ]:

tdnli.to_csv('../data/tinydata_nli_result.csv')

In [127]:
detect('75%的酒精需被用于消毒买回家的蔬菜')

买回家的蔬菜必须用75%的酒精消毒 -> 75%的酒精需被用于消毒买回家的蔬菜 :  entailment


['买回家的蔬菜必须用75%的酒精消毒', 1]

In [88]:
tdnli.loc[0,'contradiction_document']=-1

In [129]:
tdnli.head(40)

,id,label,document,entailment,contradiction,neutral,entailment_document,entailment_result,contradiction_document,contradiction_result,neutral_document,neutral_result
0,390,5,黑龙江黑河“100多人带着病毒回来了”,“100多人带着病毒回来了”发生在黑龙江黑河,黑龙江黑河没有“100多人带着病毒回来了”,重走抗联路!吉林省委党史研究室率专家一行实地踏查老黑河,黑龙江黑河“100多人带着病毒回来了”,1,黑龙江黑河“100多人带着病毒回来了”,0,黑龙江黑河“100多人带着病毒回来了”,2
1,241,7,钟南山预测“5月疫情将全面爆发”,“5月疫情将全面爆发”被钟南山预测,钟南山没有预测“5月疫情将全面爆发”,八旬妻子球场上边投篮边等钟南山回家,钟南山预测“5月疫情将全面爆发”,1,钟南山预测“5月疫情将全面爆发”,0,钟南山预测“5月疫情将全面爆发”,2
2,867,7,病毒会漂浮在空气中形成“病毒云”,“病毒云”是由漂浮在空气中的病毒形成,病毒不会漂浮在空气中形成“病毒云”,米兰官方:皮奥利新冠病毒检测呈阳性,病毒会漂浮在空气中形成“病毒云”,1,病毒会漂浮在空气中形成“病毒云”,0,病毒会漂浮在空气中形成“病毒云”,2
3,64,7,北京五棵松华熙“沦陷”,“沦陷”发生在北京五棵松华熙,北京五棵松华熙没有“沦陷”,11月北京哪里的房子有潜力?看海淀五棵松房价走势,北京五棵松华熙“沦陷”,1,北京五棵松华熙“沦陷”,0,北京五棵松华熙“沦陷”,2
4,209,4,“五一”离开温州回来需自费做核酸检测,“五一”离开温州回来需自己出钱做核酸检测,“五一”离开温州回来不需要自费做核酸检测,浙江温州皮革厂倒闭了,“五一”离开温州，回来需自费做核酸检测,1,“五一”离开温州，回来需自费做核酸检测,0,“五一”离开温州回来需自费做核酸检测,0
5,722,7,长颈鹿充气服公鸡服……“硬核防护服”真有用,“硬核防护服”真有用长颈鹿充气服公鸡服,长颈鹿充气服公鸡服……“硬核防护服”真没用,"长颈鹿交配时一方会先喝尿?这是独特的爱好,还是另有原因!",长颈鹿充气服、公鸡服……“硬核防护服”真有用,1,长颈鹿充气服、公鸡服……“硬核防护服”真有用,0,长颈鹿充气服公鸡服……“硬核防护服”真有用,2
6,962,8,美 14 万流感死者或部分死于新冠,14 万流感死者或部分死于新冠在美国,美 14 万流感死者或没有死于新冠,云南省疾控中心提醒:防范流感需及时接种疫苗,美 1.4 万流感死者或部分死于新冠,1,美 1.4 万流感死者或部分死于新冠,0,美 14 万流感死者或部分死于新冠,2
7,1013,3,15 秒未戴口罩接触病患被感染,15 秒未戴口罩接触病患有被感染的风险,15 秒未戴口罩接触病患不会被感染,疫情期间不戴口罩很不安全,15 秒未戴口罩接触病患被感染,1,15 秒未戴口罩接触病患被感染,0,15 秒未戴口罩接触病患被感染,2
8,21,6,宁波高新区新增1例确诊病例,1例确诊病例在宁波高新区新增,宁波高新区没有新增1例确诊病例,"宁波70后富豪香港被砍,神秘发家史成谜",宁波高新区新增1例确诊病例,1,宁波高新区新增1例确诊病例,0,宁波高新区新增1例确诊病例,2
9,42,7,广西一酒店接待北京客人致20多人发烧被隔离,20多人发烧被隔离由于广西一酒店接待北京客人,广西一酒店接待北京客人并没有致20多人发烧被隔离,天气将迎转折!@广西人 降雨订单即将到货 请注意签收,广西一酒店接待北京客人致20多人发烧被隔离,1,广西一酒店接待北京客人致20多人发烧被隔离,0,广西一酒店接待北京客人致20多人发烧被隔离,2
